In [1]:
from datetime import datetime, timedelta
import os
import configparser
import requests
import pandas as pd
import numpy as np
import boto3
import io

In [5]:
# Read config files to get file locations
config = configparser.RawConfigParser()
config.read('../configs/param.cfg')
path_question = config['FILE_LOCATION'][f'loc_so_question']
path_question_tag = config['FILE_LOCATION'][f'loc_so_question_tag']

In [7]:
# Read downloaded questions file as DataFrame
df_question = pd.read_csv(path_question, compression='zip')

In [9]:
# Question file profiling
desc = df_question.describe()
desc.loc['count'] = desc.loc['count'].astype(int).astype(str)
desc.iloc[1:] = desc.iloc[1:].applymap('{:.6f}'.format)
print(desc)

                    Id         Score     OwnerUserId AnswerCount
count         17203824      17203824        13088517    15463823
mean   22655681.906628      1.204167  2274603.883636    1.417859
std    11753988.576403     15.645736  1911360.680041    1.498361
min           1.000000   -154.000000        1.000000   -5.000000
25%    12841969.750000      0.000000   692377.000000    1.000000
50%    23316485.500000      0.000000  1697528.000000    1.000000
75%    32817102.250000      1.000000  3546476.000000    2.000000
max    42069071.000000  16902.000000  7520803.000000  518.000000


In [11]:
# Read downloaded question_tags file as DataFrame
df_question_tag = pd.read_csv(path_question_tag, compression='zip')

In [12]:
# Question_tags file profiling
desc = df_question_tag.describe()
desc.loc['count'] = desc.loc['count'].astype(int).astype(str)
desc.iloc[1:] = desc.iloc[1:].applymap('{:.6f}'.format)
print(desc)

                    Id
count         50576842
mean   22799501.591920
std    11701022.581574
min           1.000000
25%    13074746.000000
50%    23503290.000000
75%    32885768.750000
max    42069071.000000
